In [1]:
__author__ = 'Team_4'
__Date__ = 'December 2018'

## Load Modules

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
__dir__ = os.getcwd()[:-11]

from chainer import Chain, Variable, optimizers
import chainer.functions as F
import chainer.links as L

from gensim.models.keyedvectors import KeyedVectors

## Define The three networks

In [3]:
class First_Network(Chain):
    def __init__(self, in_size, hidden_size, out_size):
        super(First_Network, self).__init__(
            hh = L.Linear(in_size, hidden_size),
            hy = L.Linear(hidden_size, out_size),
        )

    def __call__(self, x):
        x = Variable(x)
        h = self.hh(x)
        h = F.dropout(h,0.3)
        h = F.relu(h)
        y = self.hy(h)
        y = F.softmax(y)

        return y


class Second_Network(Chain):
    def __init__(self,vocab_size, in_size, out_size):
        super(Second_Network, self).__init__(
            xh=L.EmbedID(vocab_size, in_size),
            hh=L.LSTM(in_size, out_size),
        )

    def forward(self, x):
        x = Variable(x)
        x = self.xh(x)
        if self.i == 0:
            self.x = x
        y = self.hh(x)

        return y

    def __call__(self,word):
        self.reset()
        self.i = 0
        if word == []:
            return self.forward(np.array([0],dtype=np.int32))
        
        for char in word:
            out = self.forward(char)
            self.i += 1

        return out

    def reset(self):
        self.hh.reset_state()


class Third_Network(Chain):
    def __init__(self, in_size, hidden_size, out_size):
        super(Third_Network, self).__init__(
            hh1 = L.Linear(in_size, hidden_size),
            hh2 =L.Linear(in_size, hidden_size),
            hy = L.Linear(hidden_size*2, out_size),
        )

    def __call__(self, x1,x2,t):
        t = Variable(t)
        h1 = self.hh1(x1)
        h2 = self.hh2(x2)
        h1 = F.dropout(h1,0.3)
        h2 = F.dropout(h2, 0.3)
        h1 = F.relu(h1)
        h2 = F.relu(h2)
        h = F.concat([h1, h2])
        out = self.hy(h)
        out = F.tanh(out)
        out = F.normalize(out)

        return F.mean_squared_error(out,t)

In [9]:
def word_to_index(word):
    word_index = [ord (char) - 97 for char in word]
    return word_index

In [5]:
dic = KeyedVectors.load_word2vec_format(__dir__ + 'data/glove.6B.100d.bin')
original_word = 'created'
glove_vec = dic.get_vector(original_word).reshape(1,-1)

In [8]:
first_net = First_Network(100,30,len(original_word))
second_net = Second_Network(27,30, 50)
second_net2 = Second_Network(27, 30, 50)
third_net = Third_Network(50, 50, 100)

optimizer1 = optimizers.Adam()
optimizer1.setup(first_net)

optimizer2 = optimizers.Adam()
optimizer2.setup(first_net)

optimizer3 = optimizers.Adam()
optimizer3.setup(first_net)

optimizer4 = optimizers.Adam()
optimizer4.setup(first_net)

In [ ]:
loss_record = []
f1_record = []
    for i in range(50):
        first_net.cleargrads()
        second_net.cleargrads()
        second_net2.cleargrads()
        third_net.cleargrads()
        
        f1 = first_net(glove_vec)
        split_ix = np.argmax(f1.data,axis=1) + 1

        word1 = original_word[:np.int(split_ix)]
        word2 = original_word[np.int(split_ix):]

        vec1 = [np.array([ord(char) - 96], dtype=np.int32) for char in word1]
        y1 = second_net(vec1)

        vec2 = [np.array([ord(char) - 96], dtype=np.int32) for char in word2]
        y2 = second_net2(vec2)


        loss = third_net(y1,y2,glove_vec)
        loss_record.append(float(loss.data))
        loss.backward(retain_grad=True)


        optimizer4.update()
        optimizer3.update()
        optimizer2.update()

        f1_loss = F.concat([second_net.x.grad,second_net2.x.grad])
        f1_loss = F.sum(F.absolute(f1_loss))
        f1_record.append(float(f1_loss.data))
        f1.grad = (f1 * f1_loss.data).data
        #f1.grad = (f1 * loss.data).data
        f1.unchain_backward()
        f1.backward(retain_grad=True)
        optimizer1.update()